Process Text from entire books
===

* This could be the first step of a possible final project  
* In a further step text could be analyzed   
* Books could be rated according to certain dimensions  
* Find the music with the most similar lyrics to a book  

## From PDF to Text

## From Ebook to Text

## Books that were already converted from to txt with calibre

In [121]:
from glob import glob

books = glob("data/*.txt")

books

['data/Thomas Pynchon - V.txt',
 "data/Thomas Pynchon - Gravity's rainbow.txt",
 'data/William Faulkner - The Sound and the Fury.txt']

In [122]:
with open('data/William Faulkner - The Sound and the Fury.txt', "r") as fo:
    book = fo.readlines()

print(f"Book has {len(book)} paragraphs.")

Book has 6664 paragraphs.


### Inspection
1. Novel starts at paragraph 58
2. There are meaningless numbers in the format "1.8" in between the text on single lines.
3. Each paragraph ends with `\n` which is not necessary here
4. There are entirely empty lines

#### 1. Trim

In [123]:
book = book[58:]

#### 2. Remove meaningless numbers and 3. remove `\n`

*Example*

In [124]:
book[22]

'4.1\n'

In [125]:
book[0]

'April 7, 1928\n'

In [126]:
import regex as re

rm_pattern_1 = "^[0-9]*\.[0-9]*\\n$"
rm_pattern_2 = "\\n$"

for i, _ in enumerate(book):
    book[i] = re.sub(rm_pattern_1, "", book[i]) 
    book[i] = re.sub(rm_pattern_2, "", book[i]) 

*Example*

In [127]:
book[22]

''

In [128]:
book[0]

'April 7, 1928'

#### 4. Remove empty lines

*Before*

In [134]:
for p in book[:7]:
    print(p)

April 7, 1928



Through the fence, between the curling flower spaces, I could see them hitting. They were coming toward where the flag was and I went along the fence. Luster was hunting in the grass by the flower tree. They took the flag out, and they were hitting. Then they put the flag back and they went to the table, and he hit and the other hit. Then they went on, and I went along the fence. Luster came away from the flower tree and we went along the fence and they stopped and we stopped and I looked through the fence while Luster was hunting in the grass.

"Here, caddie." He hit. They went away across the pasture. I held to the fence and watched them going away.


*After*

In [175]:
book_trimmed = [p for p in book if p != '']

for p in book_trimmed[:3]:
    print(p)

April 7, 1928
Through the fence, between the curling flower spaces, I could see them hitting. They were coming toward where the flag was and I went along the fence. Luster was hunting in the grass by the flower tree. They took the flag out, and they were hitting. Then they put the flag back and they went to the table, and he hit and the other hit. Then they went on, and I went along the fence. Luster came away from the flower tree and we went along the fence and they stopped and we stopped and I looked through the fence while Luster was hunting in the grass.
"Here, caddie." He hit. They went away across the pasture. I held to the fence and watched them going away.


## Start analyzing the text

### Fix contractions & lower case

In [176]:
import contractions


for i, _ in enumerate(book_trimmed):
    book_trimmed[i] = contractions.fix(book_trimmed[i]).lower()

### Tokenize paragraphs to sentences

In [177]:
import nltk
from copy import deepcopy

book_trimmed_tokenized = deepcopy(book_trimmed)

for i, p in enumerate(book_trimmed):
    book_trimmed_tokenized[i] = nltk.sent_tokenize(p)

### Sentiment scores for sentences

In [189]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

<div class="alert alert-warning">Let's see what is the difference between computing the score over the individual sentences or for the whole paragraph. It seems that this is an absolute value that accumulates the longer a text. High scores for long texts can thus be misleading. Is there a relative measure? Dividing by number of words or sentences? That would be necessary to make valid comparisons. I.e. 2 books could have both positive scores, but one has higher scores. Could it be because of its length?</div>

In [212]:
idx = 6

print("A) whole paragraph:\n")
print(book_trimmed[idx])
print("=>", sid.polarity_scores(book_trimmed[idx])['compound'])

print("\nB) single sentences:\n")
for sent in book_trimmed_tokenized[idx]:
    print(sent)
    print("=>", sid.polarity_scores(sent)['compound'])

A) whole paragraph:

it was red, flapping on the pasture. then there was a bird slanting and tilting on it. luster threw. the flag flapped on the bright grass and the trees. i held to the fence.
=> 0.4404

B) single sentences:

it was red, flapping on the pasture.
=> 0.0
then there was a bird slanting and tilting on it.
=> 0.0
luster threw.
=> 0.0
the flag flapped on the bright grass and the trees.
=> 0.4404
i held to the fence.
=> 0.0


### Numbers to words

<div class="alert alert-warning">Do we need the this?</div>

In [ ]:
import inflect
inf_engine =  inflect.engine()
for i, _ in enumerate(book_trimmed):
    book_trimmed[i] = inf_engine.number_to_words(book_trimmed[i])